In [1]:
import re
import time
import json
import lxml
import requests
import urllib.request
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait

In [2]:
options = Options()
chromedriver_dir = '/home/imax/airbnb/ChromeDriver/chromedriver'
driver = webdriver.Chrome(chromedriver_dir, chrome_options=options)
driver.set_window_size(1924, 1200)

/home/imax/.pyenv/versions/fc-airbnb/lib/python3.6/site-packages/ipykernel_launcher.py:3: DeprecationWarning: use options instead of chrome_options
  This is separate from the ipykernel package so we can avoid doing imports until


In [3]:
api_url = ['https://www.airbnb.co.kr/api/v2/explore_tabs?version=1.3.9&satori_version=1.1.0&_format=for_explore_search_web&experiences_per_grid=20&items_per_grid=18&guidebooks_per_grid=20&auto_ib=true&fetch_filters=true&has_zero_guest_treatment=true&is_guided_search=true&is_new_cards_experiment=true&luxury_pre_launch=true&query_understanding_enabled=true&show_groupings=true&supports_for_you_v3=true&timezone_offset=540&client_session_id=8065bab3-9e7f-431c-ac32-7f853521d812&metadata_only=false&is_standard_search=true&tab_id=home_tab&section_offset=7&items_offset=0&recommendation_item_cursor=&refinement_paths%5B%5D=%2Fhomes&place_id=ChIJzWXFYYuifDUR64Pq5LTtioU&allow_override%5B%5D=&s_tag=Is5D9pTY&last_search_session_id=7dec65e8-9baa-4888-aa8a-420a4980a23d&federated_search_session_id=58999f58-4209-4f2b-87c4-3a0a5dd44763&screen_size=large&query=%EC%84%9C%EC%9A%B8%ED%8A%B9%EB%B3%84%EC%8B%9C&_intents=p1&key=d306zoyjsyarp7ifhu67rjxn52tv0t20&currency=KRW&locale=ko']

In [4]:
def format_float(num):
    result = ('%i' if num == int(num) else '%s') % num
    return int(result)

In [6]:
room_count = 0

# api_url 리스트에 있는 주소를 하나씩 불러온다
# 너무 많으면 차단될까봐 하나씩 하도록 위에서 지정
for url in api_url:
    html = requests.get(url)
    soup = BeautifulSoup(html.text, 'lxml')
    p_tags = soup.select('p')
    data = json.loads(html.text)
    explore_tabs = data.get('explore_tabs')
    sections = explore_tabs[0].get('sections')
    
    try:
        listings = sections[1].get('listings')
    except:
        listings = sections[0].get('listings')
        
    listings_info = listings[0].get('listing')  
    
    # house_info 리스트 내 정보에 해당하는 값 출력
    for index in range(16, len(listings)):
        print(f'================={index}번 방 정보==============')
        
        house_info = ['bathrooms', 'beds', 'bedrooms', 'person_capacity', 'room_type', 'localized_city','lat', 'lng', 'name', 'room_and_property_type', 'public_address']
        house_content = []

        listings_info = listings[index].get('listing')
        for num, key in enumerate(house_info):
            house_content.append(listings_info[key])

        pricing_quote = listings[index].get('pricing_quote')
        rate = pricing_quote.get('rate')

        house_content[0] = format_float(house_content[0])
        price = format_float(rate.get('amount'))
        
        print('house_content 리스트에 값 저장 성공')
        

        listings_info = listings[index].get('listing')

        # 디테일 정보를 얻기 위해
        # URL에 방 ID를 붙여서 driver.get으로 이동
        base_url = 'https://www.airbnb.co.kr/rooms/'
        home_id = listings_info['id']
        room_url = base_url + str(home_id)
        driver.get(room_url)
        time.sleep(3)
        
        # home_info_1
        home_info_1 = ''
        try:
            html = driver.find_element_by_class_name('_1kzvqab3').get_attribute('outerHTML')
            soup = BeautifulSoup(html, 'lxml')
            div_elm = soup.select_one('div#summary').find_next_sibling('div')
            highlight = div_elm.select_one('div._uknigi').findChildren('div', recursive=False)
            for i in highlight:
                title = i.select_one('span').get_text()
                content = i.select_one('span._fgdupie').get_text()
                home_info = title + '\n' + content + '\n' + '\n'
                home_info_1 += home_info

                                
        except:
            html = driver.find_element_by_class_name('_1kzvqab3').get_attribute('outerHTML')
            soup = BeautifulSoup(html, 'lxml')
            div_elm = soup.select_one('div#summary').find_next_sibling('div')
            highlight = div_elm.select('div._n5lh69r')
            for i in highlight:               
                title = i.select_one('div._1n57hdr7').get_text()
                content = i.select_one('div._ncwphzu').get_text()
                home_info = title + '\n' + content + '\n' + '\n'
                home_info_1 += home_info
        
        print('home_info_1 생성')
        
        # home_info_2
        home_info_2 = ''
        driver.implicitly_wait(5)
        details = driver.find_element_by_id('details')
        html = details.find_elements_by_class_name('_fgdupie')
        
        for i in html:
            html = i.get_attribute('innerHTML')
            soup = BeautifulSoup(html, 'lxml')
            span = soup.select('span')
            for i in span:
                home_info = i.get_text()
                home_info_2 += home_info + '\n'
        print('home_info_2 생성')
#         html = details.find_elements_by_class_name('_fgdupie').get_attribute('innerHTML')
#         html = driver.find_element_by_id('details').get_attribute('outerHTML')
#         soup = BeautifulSoup(html, 'lxml')
#         span = soup.select_one('p._6z3til > span').select('span')
#         span = soup.select('span')
#         for i in span:
#             home_info = i.get_text()
#             home_info_2 += home_info + '\n'
#         print(home_info_2)
#         print('home_info_2')
        
        # home_info_3
        detail_infos = driver.find_element_by_id('details')
        detail_infos.find_element_by_class_name('_1k01n3v1').click()
        info = driver.find_element_by_id('details')
        html = info.find_element_by_class_name('_x01z5ll').get_attribute('outerHTML')
        soup = BeautifulSoup(html, 'lxml')
        span = soup.select('span > span')
#         sub_title = soup.select('div._1n57hdr7')
        
#         for i in sub_title:
#             print(i.get_text())
#         print('home_info_3에 들어갈 내용')
        time.sleep(3)
        
        home_info_3 = ''
        for i in span:
            home_info = i.get_text()
            home_info_3 += home_info + '\n'
        
        print('home_info_3 생성')
        time.sleep(3)
        try:
            detail_infos.find_element_by_class_name('_1k01n3v1').click()
        except:
            time.sleep(3)


        # home_info_4
        home_info_4 = ''
        html = driver.find_element_by_id('amenities').get_attribute('outerHTML')
        soup = BeautifulSoup(html, 'lxml')
        amenities = soup.select('div._ncwphzu')
        for amenity in amenities:
            home_info = amenity.get_text()
            home_info_4 += home_info + '\n'  
        print('home_info_4 생성')
        
        time.sleep(3)
        print('----------------------------------------------------------------------------------------')
     
        # DB 저장 후 Instance 생성
        room = Room.objects.create(
            bathrooms=house_content[0], 
            beds=house_content[1], 
            bedrooms=house_content[2],
            person_capacity=house_content[3],
            room_type=house_content[4],
            city=house_content[5],
            lat=house_content[6],
            lng=house_content[7],
            room_name=house_content[8],
            room_and_property_type=house_content[9],
            public_address=house_content[10],
            price=price,
            room_host=1,
            room_info_1=home_info_1,
            room_info_2=home_info_2,
            room_info_3=home_info_3,
            room_info_4=home_info_4
        )
        print('Room DB 저장 성공, 인스턴스 생성 성공')
        
        # 이미지 다운 및 DB 저장
        count = 1
        for url in listings_info.get('picture_urls'):
            home_id = listings_info['id']
            img_url = urllib.request.urlretrieve(url, f'{home_id}-{count}.jpg')
            result_url = f'picture/host/listing/{home_id}-{count}.jpg'
            RoomPhoto.objects.create(room=room, room_photo=result_url)
            if count == 5:
                break
            count += 1
        print('photo_list에 이미지 주소 모두 들어감')
        
       
        # thumbnail 사진 값을 다운하고 DB에 저장
        thumbnail = []
        host_thumbnail_url = listings_info.get('host_thumbnail_url')
        host_thumbnail_url_small = listings_info.get('host_thumbnail_url_small')
        home_id = listings_info['id']
        
        host_thumbnail = urllib.request.urlretrieve(host_thumbnail_url, f'{home_id}-thumbnail.jpg')
        thumbnail.append(f'pictures/host/{home_id}-thumbnail.jpg')
        host_thumbnail_small = urllib.request.urlretrieve(host_thumbnail_url_small, f'{home_id}-thumbnail_small.jpg')
        thumbnail.append(f'pictures/host/{home_id}-thumbnail_small.jpg')
               
        HostImages.objects.create(
            host_images=room,
            host_thumbnail_url=thumbnail[0],
            host_thumbnail_url_small=thumbnail[1],
        )
        print('HostImages DB 저장 성공')
        
        # room instance 에 amenity 추가
        amenity_list = listings_info['amenity_ids']
        for num in amenity_list:
            # info에는 id가 있으나 Amenity 모델에는 없을 경우
            try:
                amenity = Amenity.objects.get(id=num)
                room.amenities.add(amenity)
            except:
                pass
        print('room 인스턴스에 amenity 추가 성공')
        
        
        room_count += 1
        print("======================끝======================")
        print()

=================14번 방 정보==============
house_content 리스트에 값 저장 성공
home_info_1 생성
home_info_2 생성
home_info_3 생성
home_info_4 생성
----------------------------------------------------------------------------------------
Room DB 저장 성공, 인스턴스 생성 성공
photo_list에 이미지 주소 모두 들어감
HostImages DB 저장 성공
room 인스턴스에 amenity 추가 성공
======================끝======================

=================15번 방 정보==============
house_content 리스트에 값 저장 성공
home_info_1 생성
home_info_2 생성
home_info_3 생성
home_info_4 생성
----------------------------------------------------------------------------------------
Room DB 저장 성공, 인스턴스 생성 성공
photo_list에 이미지 주소 모두 들어감
HostImages DB 저장 성공
room 인스턴스에 amenity 추가 성공
======================끝======================

=================16번 방 정보==============
house_content 리스트에 값 저장 성공
home_info_1 생성
home_info_2 생성
home_info_3 생성
home_info_4 생성
----------------------------------------------------------------------------------------
Room DB 저장 성공, 인스턴스 생성 성공
photo_list에 이미지 주소 모두 들어감
HostImages D